# Intro
Inference notebook for [Hotel-ID starter - classification - traning](https://www.kaggle.com/code/michaln/hotel-id-starter-classification-traning)



# Readme
Er staan nu bij input allemaal arcmargin folders, en checkpoint folders met checkpoints files erin. Het irritante is dat Input een read-only file-system is, en je niet alle checkpoint files makkelijk in 1 folder kan stoppen. Verder als je een model traint (in train notebook), vind ik geen makkelijkere manier dan om de checkpoint file te downloaden en manueel hier te uploaden. Als je een makkelijker manier vindt, laat maar weten.

# Let op
Als je een model traint op bv 512x512 images, moet je IMG_SIZE hier op 2 plekken naar 512 zetten. Je moet verder onderin (bij model_array) naar je getrainde checkpoint verwijzen. Verder als je een ander model hebt gebruikt moet je overal waar efficientnet_b0 staat, dit vervangen door jouw model.

# Setup

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

# Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import os
import math

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight
from PIL import Image as pil_image
from tqdm import tqdm

import scipy

import matplotlib
import matplotlib.pyplot as plt
#import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import timm
from timm.optim import Lookahead, RAdam

# Global

In [ ]:
SEED = 42
IMG_SIZE = 256

PROJECT_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
TRAIN_DATA_FOLDER = "../input/hotelid-2022-train-images-256x256/images/"
TEST_DATA_FOLDER = PROJECT_FOLDER + "test_images/"

In [ ]:
print(os.listdir(PROJECT_FOLDER))

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


[17:23]
Ja 4 nog, ik wil nog proberen boven 0.65 te komen, maar heb aan 3 g# Dataset and transformations

In [ ]:
import albumentations as A
import albumentations.pytorch as APT
import cv2 

IMG_SIZE = 256

base_transform = A.Compose([
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])


test_tta_transforms = {
    "base": A.Compose([A.ToFloat(),  APT.transforms.ToTensorV2(),]),
    "crop1": A.Compose([A.RandomResizedCrop(IMG_SIZE, IMG_SIZE, scale=(0.6,1.0), p=1), A.ToFloat(),  APT.transforms.ToTensorV2(),]),
    "crop2": A.Compose([A.RandomResizedCrop(IMG_SIZE, IMG_SIZE, scale=(0.6,1.0), p=1),  APT.transforms.ToTensorV2(),]),
    "crop3": A.Compose([A.RandomResizedCrop(IMG_SIZE, IMG_SIZE, scale=(0.6,1.0), p=1),  APT.transforms.ToTensorV2(),]),
    "crop4": A.Compose([A.RandomResizedCrop(IMG_SIZE, IMG_SIZE, scale=(0.6,1.0), p=1),  APT.transforms.ToTensorV2(),]),
    "crop5": A.Compose([A.RandomResizedCrop(IMG_SIZE, IMG_SIZE, scale=(0.6,1.0), p=1),  APT.transforms.ToTensorV2(),]),
    "h_flip": A.Compose([A.ToFloat(), A.HorizontalFlip(p=1), APT.transforms.ToTensorV2(),]),
    "v_flip": A.Compose([A.ToFloat(), A.VerticalFlip(p=1), APT.transforms.ToTensorV2(),]),
    "rotate+90": A.Compose([A.ToFloat(), A.Rotate(limit=90, p=1), APT.transforms.ToTensorV2(),]),
    "rotate-90": A.Compose([A.ToFloat(), A.Rotate(limit=-90, p=1), APT.transforms.ToTensorV2(),]),
#     "rand_bright": A.Compose([A.ToFloat(), A.RandomBrightness(p=1), APT.transforms.ToTensor(),]),
}

In [ ]:
def pad_image(img):
    w, h, c = np.shape(img)
    if w > h:
        pad = int((w - h) / 2)
        img = cv2.copyMakeBorder(img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=0)
    else:
        pad = int((h - w) / 2)
        img = cv2.copyMakeBorder(img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=0)
        
    return img


def open_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = pad_image(img)
    return cv2.resize(img, (IMG_SIZE, IMG_SIZE))


class HotelImageDataset:
    def __init__(self, data, transform=None, data_folder="train_images/"):
        self.data = data
        self.data_folder = data_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        record = self.data.iloc[idx]
        image_path = self.data_folder + record["image_id"]
        
        if "test" in self.data_folder:
            image = np.array(open_and_preprocess_image(image_path)).astype(np.uint8)
        else:
            image = np.array(pil_image.open(image_path)).astype(np.uint8)

        if self.transform:
            transformed = self.transform(image=image)
        
        return {
            "image" : transformed["image"],
        }

# Inpaint Model for 1 image input

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

def extract_mask(img): 
    
#Outcommented code used to test if the operation worked and visualize what happend

# img = cv2.imread("../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/abc.jpg", cv2.IMREAD_COLOR) 
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# mask = cv2.imread("../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_masks/00000.png", cv2.IMREAD_COLOR)
# mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
# plt.imshow(img)
# plt.imshow(mask)

#find out colour of mask:
#print(mask.shape)
#print((mask[:,:,0]==0).all())

#check where the image has a certain colour
mask_index_picture = np.logical_and(np.logical_and(np.where(img[:,:,0] == 255), np.where(img[:,:,1] == 0),np.where(img[:,:,2] == 0 )
#in future: do a K-neighbour check to make sure it is a certain size

#extract mask
art_mask = np.zeros(img.shape)
art_mask[mask_index_picture] = [255, 0, 0]

return art_mask

def list2nparray(lst, dtype=None):
    """fast conversion from nested list to ndarray by pre-allocating space"""
    if isinstance(lst, np.ndarray):
        return lst
    assert isinstance(lst, (list, tuple)), "bad type: {}".format(type(lst))
    assert lst, "attempt to convert empty list to np array"
    if isinstance(lst[0], np.ndarray):
        dim1 = lst[0].shape
        assert all(i.shape == dim1 for i in lst)
        if dtype is None:
            dtype = lst[0].dtype
            assert all(i.dtype == dtype for i in lst), "bad dtype: {} {}".format(
                dtype, set(i.dtype for i in lst)
            )
    elif isinstance(lst[0], (int, float, complex, np.number)):
        return np.array(lst, dtype=dtype)
    else:
        dim1 = list2nparray(lst[0])
        if dtype is None:
            dtype = dim1.dtype
        dim1 = dim1.shape
    shape = [len(lst)] + list(dim1)
    rst = np.empty(shape, dtype=dtype)
    for idx, i in enumerate(lst):
        rst[idx] = i
    return rst


def get_img_list(path):
    if Path(path).is_file():
        return [Path(path)]
    else:
        return (
            sorted(list(Path(path).glob("*.png")))
            + sorted(list(Path(path).glob("*.jpg")))
            + sorted(list(Path(path).glob("*.jpeg")))
        )


def gen_miss(img, mask, output):

    imgs = get_img_list(img)
    masks = get_img_list(mask) #masks here is a path to the masks image
    print("Total i-mages:", len(imgs), len(masks))

    out = Path(output)
    out.mkdir(parents=True, exist_ok=True)

    for i, (img, mask) in enumerate(zip(imgs, masks)):
        path = out.joinpath("miss_%04d.png" % (i + 1))
        img = cv2.imread(str(img), cv2.IMREAD_COLOR)
        mask = cv2.imread(str(mask), 0)
        mask = cv2.resize(mask, img.shape[:2][::-1]) #here mask is resized to shape defined beforehand
        mask = mask[..., np.newaxis]
        mask *= 255
        miss = img * (mask > 127) + 255 * (mask <= 127)
        plt.imshow(miss)
        plt.show()
        cv2.imwrite(str(path), miss)


def merge_imgs(dirs, output, row=1, gap=2, res=512):

    image_list = [get_img_list(path) for path in dirs]
    img_count = [len(image) for image in image_list]
    print("Total images:", img_count)
    assert min(img_count) > 0, "Please check the path of empty folder."

    output_dir = Path(output)
    output_dir.mkdir(parents=True, exist_ok=True)

    n_img = len(dirs)
    row = row
    column = (n_img - 1) // row + 1
    print("Row:", row)
    print("Column:", column)

    for i, unit in tqdm(enumerate(zip(*image_list))):
        name = output_dir.joinpath("merge_%04d.png" % i)
        merge = (
            np.ones(
                [res * row + (row + 1) * gap, res * column + (column + 1) * gap, 3],
                np.uint8,
            )
            * 255
        )
        for j, img in enumerate(unit):
            r = j // column
            c = j - r * column
            img = cv2.imread(str(img), cv2.IMREAD_COLOR)
            if img.shape[:2] != (res, res):
                img = cv2.resize(img, (res, res))
            start_h, start_w = (r + 1) * gap + r * res, (c + 1) * gap + c * res
            merge[start_h : start_h + res, start_w : start_w + res] = img
        cv2.imwrite(str(name), merge)
def resize_like(x, target, mode="bilinear"):
    return F.interpolate(x, target.shape[-2:], mode=mode, align_corners=False)

def get_norm(name, out_channels):
    if name == 'batch':
        norm = nn.BatchNorm2d(out_channels)
    elif name == 'instance':
        norm = nn.InstanceNorm2d(out_channels)
    else:
        norm = None
    return norm


def get_activation(name):
    if name == 'relu':
        activation = nn.ReLU()
    elif name == 'elu':
        activation == nn.ELU()
    elif name == 'leaky_relu':
        activation = nn.LeakyReLU(negative_slope=0.2)
    elif name == 'tanh':
        activation = nn.Tanh()
    elif name == 'sigmoid':
        activation = nn.Sigmoid()
    else:
        activation = None
    return activation


class Conv2dSame(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super().__init__()

        padding = self.conv_same_pad(kernel_size, stride)
        if type(padding) is not tuple:
            self.conv = nn.Conv2d(
                in_channels, out_channels, kernel_size, stride, padding)
        else:
            self.conv = nn.Sequential(
                nn.ConstantPad2d(padding*2, 0),
                nn.Conv2d(in_channels, out_channels, kernel_size, stride, 0)
            )

    def conv_same_pad(self, ksize, stride):
        if (ksize - stride) % 2 == 0:
            return (ksize - stride) // 2
        else:
            left = (ksize - stride) // 2
            right = left + 1
            return left, right

    def forward(self, x):
        return self.conv(x)


class ConvTranspose2dSame(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super().__init__()

        padding, output_padding = self.deconv_same_pad(kernel_size, stride)
        self.trans_conv = nn.ConvTranspose2d(
            in_channels, out_channels, kernel_size, stride,
            padding, output_padding)

    def deconv_same_pad(self, ksize, stride):
        pad = (ksize - stride + 1) // 2
        outpad = 2 * pad + stride - ksize
        return pad, outpad

    def forward(self, x):
        return self.trans_conv(x)


class UpBlock(nn.Module):

    def __init__(self, mode='nearest', scale=2, channel=None, kernel_size=4):
        super().__init__()

        self.mode = mode
        if mode == 'deconv':
            self.up = ConvTranspose2dSame(
                channel, channel, kernel_size, stride=scale)
        else:
            def upsample(x):
                return F.interpolate(x, scale_factor=scale, mode=mode)
            self.up = upsample

    def forward(self, x):
        return self.up(x)


class EncodeBlock(nn.Module):

    def __init__(
            self, in_channels, out_channels, kernel_size, stride,
            normalization=None, activation=None):
        super().__init__()

        self.c_in = in_channels
        self.c_out = out_channels

        layers = []
        layers.append(
            Conv2dSame(self.c_in, self.c_out, kernel_size, stride))
        if normalization:
            layers.append(get_norm(normalization, self.c_out))
        if activation:
            layers.append(get_activation(activation))
        self.encode = nn.Sequential(*layers)

    def forward(self, x):
        return self.encode(x)


class DecodeBlock(nn.Module):

    def __init__(
            self, c_from_up, c_from_down, c_out, mode='nearest',
            kernel_size=4, scale=2, normalization='batch', activation='relu'):
        super().__init__()

        self.c_from_up = c_from_up
        self.c_from_down = c_from_down
        self.c_in = c_from_up + c_from_down
        self.c_out = c_out

        self.up = UpBlock(mode, scale, c_from_up, kernel_size=scale)

        layers = []
        layers.append(
            Conv2dSame(self.c_in, self.c_out, kernel_size, stride=1))
        if normalization:
            layers.append(get_norm(normalization, self.c_out))
        if activation:
            layers.append(get_activation(activation))
        self.decode = nn.Sequential(*layers)

    def forward(self, x, concat=None):
        out = self.up(x)
        if self.c_from_down > 0:
            out = torch.cat([out, concat], dim=1)
        out = self.decode(out)
        return out


class BlendBlock(nn.Module):

    def __init__(
            self, c_in, c_out, ksize_mid=3, norm='batch', act='leaky_relu'):
        super().__init__()
        c_mid = max(c_in // 2, 32)
        self.blend = nn.Sequential(
            Conv2dSame(c_in, c_mid, 1, 1),
            get_norm(norm, c_mid),
            get_activation(act),
            Conv2dSame(c_mid, c_out, ksize_mid, 1),
            get_norm(norm, c_out),
            get_activation(act),
            Conv2dSame(c_out, c_out, 1, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.blend(x)


class FusionBlock(nn.Module):
    def __init__(self, c_feat, c_alpha=1):
        super().__init__()
        c_img = 3
        self.map2img = nn.Sequential(
            Conv2dSame(c_feat, c_img, 1, 1),
            nn.Sigmoid())
        self.blend = BlendBlock(c_img*2, c_alpha)

    def forward(self, img_miss, feat_de):
        img_miss = resize_like(img_miss, feat_de)
        raw = self.map2img(feat_de)
        alpha = self.blend(torch.cat([img_miss, raw], dim=1))
        result = alpha * raw + (1 - alpha) * img_miss
        return result, alpha, raw


class DFNet(nn.Module):
    def __init__(
            self, c_img=3, c_mask=1, c_alpha=3,
            mode='nearest', norm='batch', act_en='relu', act_de='leaky_relu',
            en_ksize=[7, 5, 5, 3, 3, 3, 3, 3], de_ksize=[3]*8,
            blend_layers=[0, 1, 2, 3, 4, 5]):
        super().__init__()

        c_init = c_img + c_mask

        self.n_en = len(en_ksize)
        self.n_de = len(de_ksize)
        assert self.n_en == self.n_de, (
            'The number layer of Encoder and Decoder must be equal.')
        assert self.n_en >= 1, (
            'The number layer of Encoder and Decoder must be greater than 1.')

        assert 0 in blend_layers, 'Layer 0 must be blended.'

        self.en = []
        c_in = c_init
        self.en.append(
            EncodeBlock(c_in, 64, en_ksize[0], 2, None, None))
        for k_en in en_ksize[1:]:
            c_in = self.en[-1].c_out
            c_out = min(c_in*2, 512)
            self.en.append(EncodeBlock(
                c_in, c_out, k_en, stride=2,
                normalization=norm, activation=act_en))

        # register parameters
        for i, en in enumerate(self.en):
            self.__setattr__('en_{}'.format(i), en)

        self.de = []
        self.fuse = []
        for i, k_de in enumerate(de_ksize):

            c_from_up = self.en[-1].c_out if i == 0 else self.de[-1].c_out
            c_out = c_from_down = self.en[-i-1].c_in
            layer_idx = self.n_de - i - 1

            self.de.append(DecodeBlock(
                c_from_up, c_from_down, c_out, mode, k_de, scale=2,
                normalization=norm, activation=act_de))
            if layer_idx in blend_layers:
                self.fuse.append(FusionBlock(c_out, c_alpha))
            else:
                self.fuse.append(None)

        # register parameters
        for i, de in enumerate(self.de[::-1]):
            self.__setattr__('de_{}'.format(i), de)
        for i, fuse in enumerate(self.fuse[::-1]):
            if fuse:
                self.__setattr__('fuse_{}'.format(i), fuse)

    def forward(self, img_miss, mask):

        out = torch.cat([img_miss, mask], dim=1)

        out_en = [out]
        for encode in self.en:
            out = encode(out)
            out_en.append(out)

        results = []
        alphas = []
        raws = []
        for i, (decode, fuse) in enumerate(zip(self.de, self.fuse)):
            out = decode(out, out_en[-i-2])
            if fuse:
                result, alpha, raw = fuse(img_miss, out)
                results.append(result)
                alphas.append(alpha)
                raws.append(raw)

        return results[::-1], alphas[::-1], raws[::-1]

In [ ]:
from collections import defaultdict
from itertools import islice
from multiprocessing.pool import ThreadPool as Pool
import os
from pathlib import Path

import argparse
import cv2
import numpy as np
import torch
from tqdm import tqdm

class Tester:
    def __init__(self, model_path, input_size, batch_size):
        self.model_path = model_path
        self._input_size = input_size
        self.batch_size = batch_size
        self.init_model(model_path)

    @property
    def input_size(self):
        if self._input_size > 0:
            return (self._input_size, self._input_size)
        elif "celeba" in self.model_path:
            return (256, 256)
        else:
            return (512, 512)

    def init_model(self, path):
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
            print("Using gpu.")
        else:
            self.device = torch.device("cpu")
            print("Using cpu.")

        self.model = DFNet().to(self.device)
        checkpoint = torch.load(path, map_location=self.device)
        self.model.load_state_dict(checkpoint)
        self.model.eval()

        print("Model %s loaded." % path)

    def get_name(self, path):
        return ".".join(Path(path).name.split(".")[:-1])

    def results_path(self, output, img_path, mask_path, prefix="result"):
        img_name = self.get_name(img_path)
        mask_name = self.get_name(mask_path)
        return {
            "result_path": self.sub_dir("result").joinpath(
                "result-{}-{}.png".format(img_name, mask_name)
            ),
            "raw_path": self.sub_dir("raw").joinpath(
                "raw-{}-{}.png".format(img_name, mask_name)
            ),
            "alpha_path": self.sub_dir("alpha").joinpath(
                "alpha-{}-{}.png".format(img_name, mask_name)
            ),
        }

    def inpaint_instance(self, img, mask):
        """Assume color image with 3 dimension. CWH"""
        img = img.view(1, *img.shape)
        mask = mask.view(1, 1, *mask.shape)
        return self.inpaint_batch(img, mask).squeeze()

    def inpaint_batch(self, imgs, masks):
        """Assume color channel is BGR and input is NWHC np.uint8."""
        imgs = np.transpose(imgs, [0, 3, 1, 2])
        masks = np.transpose(masks, [0, 3, 1, 2])

        imgs = torch.from_numpy(imgs).to(self.device)
        masks = torch.from_numpy(masks).to(self.device)
        imgs = imgs.float().div(255)
        masks = masks.float().div(255)
        imgs_miss = imgs * masks
        results = self.model(imgs_miss, masks)
        if type(results) is list:
            results = results[0]
        results = results.mul(255).byte().data.cpu().numpy()
        results = np.transpose(results, [0, 2, 3, 1])
        return results

    def _process_file(self, output, img_path, mask_path):
        item = {"img_path": img_path, "mask_path": mask_path}
        item.update(self.results_path(output, img_path, mask_path))
        self.path_pair.append(item)

    def process_single_file(self, output, img_path, mask_path):
        self.path_pair = []
        self._process_file(output, img_path, mask_path)

    def process_dir(self, output, img_dir, mask_dir):
        img_dir = Path(img_dir)
        mask_dir = Path(mask_dir)
        imgs_path = sorted(list(img_dir.glob("*.jpg")) + list(img_dir.glob("*.png")))
        masks_path = sorted(list(mask_dir.glob("*.jpg")) + list(mask_dir.glob("*.png")))

        n_img = len(imgs_path)
        n_mask = len(masks_path)
        n_pair = min(n_img, n_mask)

        self.path_pair = []
        for i in range(n_pair):
            img_path = imgs_path[i % n_img]
            mask_path = masks_path[i % n_mask]
            self._process_file(output, img_path, mask_path)

    def get_process(self, input_size):
        def process(pair):
            img = cv2.imread(str(pair["img_path"]), cv2.IMREAD_COLOR)
            mask = cv2.imread(str(pair["mask_path"]), cv2.IMREAD_GRAYSCALE)
            print(mask)
            if input_size:
                img = cv2.resize(img, input_size)
                mask = cv2.resize(mask, input_size)
            img = np.ascontiguousarray(img.transpose(2, 0, 1)).astype(np.uint8)
            mask = np.ascontiguousarray(np.expand_dims(mask, 0)).astype(np.uint8)
            print(mask)

            pair["img"] = img
            pair["mask"] = mask
            return pair

        return process

    def _file_batch(self):
        pool = Pool()

        n_pair = len(self.path_pair)
        n_batch = (n_pair - 1) // self.batch_size + 1

        for i in range(n_batch):
            _buffer = defaultdict(list)
            start = i * self.batch_size
            stop = start + self.batch_size
            process = self.get_process(self.input_size)
            batch = pool.imap_unordered(process, islice(self.path_pair, start, stop))
            for instance in batch:
                for k, v in instance.items():
                    _buffer[k].append(v)
            yield _buffer

    def batch_generator(self):
        generator = self._file_batch

        for _buffer in generator():
            for key in _buffer:
                if key in ["img", "mask"]:
                    _buffer[key] = list2nparray(_buffer[key])
            yield _buffer

    def to_numpy(self, tensor):
        tensor = tensor.mul(255).byte().data.cpu().numpy()
        tensor = np.transpose(tensor, [0, 2, 3, 1])
        return tensor

    def process_batch(self, batch, output):
        imgs = torch.from_numpy(batch["img"]).to(self.device)
        masks = torch.from_numpy(batch["mask"]).to(self.device)
        imgs = imgs.float().div(255)
        masks = masks.float().div(255)
        imgs_miss = imgs * masks

        result, alpha, raw = self.model(imgs_miss, masks)
        result, alpha, raw = result[0], alpha[0], raw[0]
        result = imgs * masks + result * (1 - masks)

        result = self.to_numpy(result)
        alpha = self.to_numpy(alpha)
        raw = self.to_numpy(raw)

        for i in range(result.shape[0]):
            cv2.imwrite(str(batch["result_path"][i]), result[i])
            cv2.imwrite(str(batch["raw_path"][i]), raw[i])
            cv2.imwrite(str(batch["alpha_path"][i]), alpha[i])

    @property
    def root(self):
        return Path(self.output)

    def sub_dir(self, sub):
        return self.root.joinpath(sub)

    def prepare_folders(self, folders):
        for folder in folders:
            Path(folder).mkdir(parents=True, exist_ok=True)

    def inpaint(self, output, img, mask, merge_result=False):

        self.output = output
        self.prepare_folders(
            [self.sub_dir("result"), self.sub_dir("alpha"), self.sub_dir("raw")]
        )

        if os.path.isfile(img) and os.path.isfile(mask):
            if img.endswith((".png", ".jpg", ".jpeg")):
                self.process_single_file(output, img, mask)
                _type = "file"
            else:
                raise NotImplementedError()
        elif os.path.isdir(img) and os.path.isdir(mask):
            self.process_dir(output, img, mask)
            _type = "dir"
        else:
            print("Img: ", img)
            print("Mask: ", mask)
            raise NotImplementedError("img and mask should be both file or directory.")

        print("# Inpainting...")
        print("Input size:", self.input_size)
        for batch in self.batch_generator():
            self.process_batch(batch, output)
        print("Inpainting finished.")

        if merge_result:
            miss = self.sub_dir("miss")
            merge = self.sub_dir("merge")

            print("# Preparing input images...")
            gen_miss(img, mask, miss)
            print("# Merging...")
            merge_imgs(
                [
                    miss,
                    self.sub_dir("raw"),
                    self.sub_dir("alpha"),
                    self.sub_dir("result"),
                    img,
                ],
                merge,
                res=self.input_size[0],
            )
            print("Merging finished.")


tester = Tester('../input/df-model-places2/model_places2.pth', 512, 16)


In [ ]:
# example image
im = '../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/100055/000003766.jpg'
ms = '../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_masks/00002.png'
out = ''
tester.inpaint('./test.png', im, ms, True)
# python test.py --model model/model_celeba.pth --img samples/celeba/img --mask samples/celeba/mask --output output/celeba --merge

In [ ]:
print(os.listdir('test.png/result'))

import imageio
im = imageio.imread('./test.png/result/result-000003766-00002.png')

plt.figure()
plt.imshow(im)
plt.show()

# Model

In [ ]:
# source: https://github.com/ronghuaiyang/arcface-pytorch/blob/master/models/metrics.py
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s

        return output

class HotelIdModel(nn.Module):
    def __init__(self, out_features, embed_size=256, backbone_name="densenet161"):
        super(HotelIdModel, self).__init__()

        self.embed_size = embed_size
        self.backbone = timm.create_model(backbone_name, pretrained=False)
        in_features = self.backbone.get_classifier().in_features

        fc_name, _ = list(self.backbone.named_modules())[-1]
        if fc_name == 'classifier':
            self.backbone.classifier = nn.Identity()
        elif fc_name == 'head.fc':
            self.backbone.head.fc = nn.Identity()
        elif fc_name == 'head.flatten':
            self.backbone.head.fc = nn.Identity()
        elif fc_name == 'fc':
            self.backbone.fc = nn.Identity()
        else:
            raise Exception("unknown classifier layer: " + fc_name)

        self.arc_face = ArcMarginProduct(self.embed_size, out_features, s=30.0, m=0.50, easy_margin=False)

        self.post = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(in_features, self.embed_size*2), dim=None),
            nn.BatchNorm1d(self.embed_size*2),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(self.embed_size*2, self.embed_size)),
            nn.BatchNorm1d(self.embed_size),
        )

        print(f"Model {backbone_name} ArcMarginProduct - Features: {in_features}, Embeds: {self.embed_size}")
        
    def forward(self, input, targets = None):
        x = self.backbone(input)
        x = x.view(x.size(0), -1)
        x = self.post(x)
        
        if targets is not None:
            logits = self.arc_face(x, targets)
            return logits
        
        return x

In [ ]:
class EmbeddingNet(nn.Module):
    def __init__(self, n_classes=100, embed_size=64, backbone_name="efficientnet_b0"):
        super(EmbeddingNet, self).__init__()
        
        self.embed_size = embed_size
        self.backbone = timm.create_model(backbone_name, pretrained=False)
        in_features = self.backbone.get_classifier().in_features

        fc_name, _ = list(self.backbone.named_modules())[-1]
        if fc_name == 'classifier':
            self.backbone.classifier = nn.Identity()
        elif fc_name == 'head.fc':
            self.backbone.head.fc = nn.Identity()
        elif fc_name == 'fc':
            self.backbone.fc = nn.Identity()
        else:
            raise Exception("unknown classifier layer: " + fc_name)
        
        self.post = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(in_features, self.embed_size*2), dim=None),
            nn.BatchNorm1d(self.embed_size*2),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(self.embed_size*2, self.embed_size)),
        )

        self.classifier = nn.Sequential(
            nn.BatchNorm1d(self.embed_size),
            nn.Dropout(0.2),
            nn.Linear(self.embed_size, n_classes),
        )
        
        print(f"Model {backbone_name} EmbeddingNet - Features: {in_features}, Embeds: {self.embed_size}")
        
    def embed_and_classify(self, x):
        x = self.forward(x)
        return x, self.classifier(x)

    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        x = self.post(x)
        return x

# Model helper functions

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_embeds(loader, model, bar_desc="Generating embeds"):
    outputs_all = []
    
    model.eval()
    with torch.no_grad():
        t = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(t):
            input = sample['image'].to(args.device)
            output = model(input)
            outputs_all.extend(output.detach().cpu().numpy())
#             outputs_all.extend(output.detach().cpu().numpy().astype(np.float16))
            
            
    return outputs_all

In [ ]:
def get_distances(input, base_embeds, model_array):
    distances = None
    for i, model in enumerate(model_array):
        output = model(input)
        output = output.detach().cpu().numpy()
#         output = output.detach().cpu().numpy().astype(np.float16)
        model_base_embeds = base_embeds[i]
        output_distances = cosine_similarity(output, model_base_embeds)
        
        if distances is None:
            distances = output_distances
        else:
            distances = distances * output_distances
            
    return distances
    

def predict(loader, base_df, base_embeds, model_array, n_matches=5, bar_desc="Generating embeds"):
    preds = []
    with torch.no_grad():
        t = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(t):
            input = sample['image'].to(args.device)
            distances = get_distances(input, base_embeds, model_array)
            
            for j in range(len(distances)):
                tmp_df = base_df.copy()
                tmp_df["distance"] = distances[j]
                tmp_df = tmp_df.sort_values(by=["distance", "hotel_id"], ascending=False).reset_index(drop=True)
                preds.extend([tmp_df["hotel_id"].unique()[:n_matches]])

    return preds

def find_closest_match(args, test_loader, base_loader, model_array, n_matches=5):
    base_embeds = {}
    for i, model in enumerate(model_array):
        base_embeds[i] = get_embeds(base_loader, model, "Generating embeds for train")
    
    preds = predict(test_loader, base_loader.dataset.data, base_embeds, model_array, n_matches, f"Generating predictions")
        
    return preds

def transform(original_image):
    tensor_to_numpy = originalimage[0]["image"].numpy() 
    switched_channels = np.swapaxes(tensor_to_numpy, 0, 2)
    return switched_channels

def get_mask(original image):
    img = transform(original_image)
    
    
    

# Prepare data

In [ ]:
#test_df = pd.DataFrame(data={"image_id": os.listdir(TEST_DATA_FOLDER), "hotel_id": ""}).sort_values(by="image_id")
data_df = pd.read_csv("../input/hotelid-2022-train-images-256x256/train.csv")
sample_submission_df = pd.read_csv(PROJECT_FOLDER + "sample_submission.csv")
test_df = pd.DataFrame(data={"image_id": os.listdir(TEST_DATA_FOLDER), "hotel_id": ""}).sort_values(by="image_id")

In [ ]:
# code hotel_id mapping created in training notebook by encoding hotel_ids
hotel_id_code_df = pd.read_csv('../input/hotelcodemapping-org/hotel_id_code_mapping.csv')
hotel_id_code_map = hotel_id_code_df.set_index('hotel_id_code').to_dict()["hotel_id"]

# Prepare model

In [ ]:
def get_model(model_type, backbone_name, embed_size, checkpoint_path, args):
    if model_type == 'arcmargin':
        model = HotelIdModel(3116, embed_size, backbone_name)
    else:
        model = EmbeddingNet(3116, embed_size, backbone_name)
        
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint["model"])
    model = model.to(args.device)
    
    return model

In [ ]:
class args:
    batch_size = 32
    num_workers = 2
    n_classes = data_df["hotel_id"].nunique()
    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    
    
seed_everything(seed=SEED)

base_dataset = HotelImageDataset(data_df, base_transform, data_folder=TRAIN_DATA_FOLDER)
base_loader = DataLoader(base_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)

test_dataset = HotelImageDataset(test_df, test_tta_transforms["base"], data_folder=TEST_DATA_FOLDER)
test_dataset_paint = tester.inpaint(transform())
test_loader = DataLoader(test_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)

imagetonump = (test_dataset[0]['image'].numpy())
print(imagetonump.shape)

#test_dataset1 = HotelImageDataset(test_df, test_tta_transforms["base"], data_folder=TEST_DATA_FOLDER)
#test_loader1 = DataLoader(test_dataset1, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)
#test_dataset2 = HotelImageDataset(test_df, test_tta_transforms["h_flip"], data_folder=TEST_DATA_FOLDER)
#test_loader2 = DataLoader(test_dataset2, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)
#test_dataset3 = HotelImageDataset(test_df, test_tta_transforms["crop3"], data_folder=TEST_DATA_FOLDER)
#test_loader3 = DataLoader(test_dataset3, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)
#test_dataset4 = HotelImageDataset(test_df, test_tta_transforms["crop4"], data_folder=TEST_DATA_FOLDER)
#test_loader4 = DataLoader(test_dataset4, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)
#test_dataset5 = HotelImageDataset(test_df, test_tta_transforms["crop5"], data_folder=TEST_DATA_FOLDER)
#test_loader5 = DataLoader(test_dataset5, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)

timm.list_models(pretrained=True)

In [ ]:
model_array = [get_model("arcmargin", 
                         "eca_nfnet_l0", 4096,
                         "../input/cp-ecanfnet-256-ep6/checkpoint-arcmargin-model-eca_nfnet_l0-256x256-4096embeds-3116hotels-6.pt", 
                         args),
              ]
              
              #get_model("arcmargin", 
                         #"efficientnet_b2", 4096,
                         #"../input/arcmargin512-effb2-ep9/checkpoint-arcmargin-model-efficientnet_b2-512x512-4096embeds-3116hotels-9.pt", 
                         #args),
              #]

# Submission

In [ ]:
#%%time

#preds = predict(test_loader, model)
# replace classes with hotel_id using mapping created in trainig notebook
#preds = [[hotel_id_code_map[b] for b in a] for a in preds]
# transform array of hotel_ids into string
#test_df["hotel_id"] = [str(list(l)).strip("[]").replace(",", "") for l in preds]

#test_df.to_csv("submission.csv", index=False)
#test_df.head()
#%%time

if len(test_df) > 3:
    preds = find_closest_match(args, test_loader, base_loader, model_array, n_matches = 5)
    #df_1 = find_closest_match(args, test_loader1, base_loader, model_array, n_matches=5)
    #print(df_1[:5])
    #df_2 = find_closest_match(args, test_loader1, base_loader, model_array, n_matches=5)
    #print(df_2[:5])
    #df3 = df_1.merge(df_2, on = 'image_id')
    #print(df3[:5])
    #df_2 = find_closest_match(args, test_loader1, base_loader, model_array, n_matches=5)
    #df_2 = find_closest_match(args, test_loader1, base_loader, model_array, n_matches=5)
    #df_2 = find_closest_match(args, test_loader1, base_loader, model_array, n_matches=5)
    #preds2, distances2 = find_closest_match(args, test_loader2, base_loader, model_array, n_matches=5)
    #preds3, distances3 = find_closest_match(args, test_loader3, base_loader, model_array, n_matches=5)
    #preds4, distances4 = find_closest_match(args, test_loader4, base_loader, model_array, n_matches=5)
    #preds5, distances5 = find_closest_match(args, test_loader5, base_loader, model_array, n_matches=5)
    test_df["hotel_id"] = [str(list(l)).strip("[]").replace(",", "") for l in preds]

test_df.to_csv("submission.csv", index=False)
test_df.head()

In [ ]:
#preds = find_closest_match(args, test_loader, base_loader, model_array, n_matches=5)

#test_df["hotel_id_pred"] = [str(list(l)).strip("[]").replace(",", "") for l in preds]

#y = np.repeat([test_df["hotel_id"]], repeats=5, axis=0).T
#preds = np.array(preds)

#acc_top_1 = (preds[:, 0] == test_df["hotel_id"]).mean()
#acc_top_5 = (preds == y).any(axis=1).mean()

#print(f"Accuracy: {acc_top_1:0.4f}, top 5 accuracy: {acc_top_5:0.4f}")